In [1]:
## personal imports
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime

from fund.fund import Fund

## risk and return class imports
from returns.products import Products as returns_products_class

## standard imports
from statistics import variance
import math
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
market = Market()

In [3]:
delta_fund = Fund()
delta_fund.initialize()

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [5]:
analysis = []
for strategy in tqdm(delta_fund.strategies):
    try:
        parameters = strategy.pull_iterations().to_dict("records")
        for parameter in tqdm(parameters):
            try:
                trade = strategy.db.query("trades",{"iteration":parameter["iteration"]})
                if trade.index.size > 0:
                    positions = int(strategy.positions) if strategy.pricer_class.asset_class.value == "stocks" else 1
                    pricer_timeframe =  strategy.pricer_class.time_horizon_class.naming_convention
                    trade = trade[trade["position"] <= positions]
                    ledger = trade.sort_values("date").copy()
                    ledger = ledger[(ledger["date"]>=delta_fund.start) & (ledger["date"]<=delta_fund.end)]
                    anas = strategy.analysis
                    if ledger.index.size > 0:
                        cumulative = anas.trade_analysis(ledger,positions)
                        for key in parameter.keys():
                            cumulative[key] = parameter[key]
                        cumulative["strategy"] = strategy.name
                        analysis.append(cumulative[list(parameter.keys()) + ["strategy","pv"]].iloc[-1])
            except Exception as e:
                print(str(e))
                continue
        strategy.db.disconnect()
    except Exception as e:
        print(str(e))
a = pd.DataFrame([x.to_dict() for x in analysis])

 35%|█████████████████████████████████████████████▋                                                                                    | 45/128 [00:09<00:20,  4.12it/s]


 72%|█████████████████████████████████████████████████████████████████████████████████████████████▍                                    | 92/128 [00:20<00:08,  4.09it/s]


  7%|█████████▏                                                                                                                         | 9/128 [00:01<00:23,  5.02it/s]


 44%|████████████████████████████████████████████████████████▉                                                                         | 56/128 [00:12<00:17,  4.21it/s]


 80%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                         | 103/128 [00:23<00:06,  4.11it/s]


...

 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▊                       | 105/128 [00:06<00:01, 17.53it/s]


 61%|███████████████████████████████████████████████████████████████████████████████▏                                                  | 78/128 [00:04<00:02, 17.88it/s]


 49%|███████████████████████████████████████████████████████████████▉                                                                  | 63/128 [00:03<00:02, 22.23it/s]


 26%|█████████████████████████████████▌                                                                                                | 33/128 [00:00<00:01, 51.96it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [03:45<00:00, 12.53s/it]


In [6]:
# a.sort_values("pv",ascending=False).head(20)

In [16]:
top_by_strategy = a.sort_values("pv",ascending=False).groupby("strategy").nth(0).reset_index().sort_values("pv",ascending=False)

,strategy,constituent,value,ceiling,classification,rank,floor,buy_day,iteration,pv
1,strategy_dsrp_qsrc_qser_b_r,500,False,1,True,True,-10,1,112,87.352231
0,strategy_dsrp_qsrc_qsdr_b_r,500,False,10,True,False,-10,1,101,48.621154
4,strategy_dswp_qsrc_qser_b_r,500,True,10,False,True,0,1,91,8.579522
3,strategy_dswp_qsrc_qsdr_b_r,100,True,1,False,False,-10,1,28,6.439386
13,strategy_wsrp_qsrc_qser_b_r,500,True,10,False,False,-10,1,77,5.918024
12,strategy_wsrp_qsrc_qsdr_b_r,500,True,10,False,False,0,1,95,5.918024
11,strategy_qswp_qsrc_qsrr_b_r,500,False,10,True,False,-10,1,117,5.707648
8,strategy_qsrp_qsrc_qsrr_b_r,500,False,10,True,False,-10,1,117,5.513674
2,strategy_dsrp_qsrc_qsrr_b_r,500,False,10,True,False,-10,1,101,4.277933
6,strategy_qsrp_qsrc_qsdr_b_r,500,True,1,False,False,0,1,94,4.228364


In [8]:
for row in top_by_strategy.iterrows():
    algo_name = row[1]["strategy"]
    parameter = row[1]
    db = ADatabase(algo_name)
    db.connect()
    db.drop("optimal")
    db.store("optimal",pd.DataFrame([row[1].to_dict()]))
    db.disconnect()
    db.cloud_connect()
    db.drop("optimal")
    db.store("optimal",pd.DataFrame([row[1].to_dict()]))
    db.disconnect()

In [9]:
cumulatives = []
ledgers = []
for row in top_by_strategy.iterrows():
    name = row[1]["strategy"]
    iteration = row[1]["iteration"]
    db = ADatabase(name)
    db.connect()
    analysis = db.query("trades",{"iteration":int(iteration)})
    db.disconnect()
    ledger = analysis[["date","ticker","actual_returns","position"]]
    ledger = ledger[(ledger["date"]>=delta_fund.start) & (ledger["date"]<=delta_fund.end)]
    ledger = ledger.merge(sp500[["ticker","GICS Sector","Security"]],on="ticker",how="left")
    ledger["strategy"] = name
    ledgers.append(ledger)
    portfolio = ledger.pivot_table(index="date",columns="position",values="actual_returns").fillna(1).reset_index()
    positions = ledger["position"].max()
    cumulative = anas.iteration_analysis(portfolio,positions)
    cumulative["strategy"] = name
    for i in range(positions):
        if i not in cumulative.columns:
            cumulative[i] = 1
        cumulative.rename(columns={i:str(i)},inplace=True)
    db.cloud_connect()
    db.drop("performance")
    db.store("performance",cumulative)
    db.disconnect()
    cumulatives.append(cumulative)

In [10]:
portfolio = pd.concat(cumulatives).pivot_table(index="date",columns="strategy",values="pv").fillna(method="ffill").fillna(1).reset_index()
portfolio["pv"] = [sum([row[1][strategy]*(1/top_by_strategy.index.size) for strategy in portfolio.columns if strategy not in ["date"]]) for row in portfolio.iterrows()]

In [11]:
# portfolio[["date","pv"]].tail(1)

In [12]:
# plt.plot(portfolio["date"],portfolio["pv"])

In [13]:
# all_ledgers = pd.concat(ledgers)

In [14]:
# all_ledgers.groupby(["strategy","GICS Sector"]).mean().reset_index().sort_values("actual_returns",ascending=False)